In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data


In [3]:
###清楚数据缓存
tf.reset_default_graph()
##载入数据集
mnist= input_data.read_data_sets('MNIST_data/',one_hot=True)

###输入图片是28*28
n_inputs = 28 ###输入的列数
max_time = 28 ###输入的行数
lstm_size = 100 ###LSTM的数量
n_classes = 10 ###输出的分类数
batch_size = 50 ###每次采样的数量
n_batch = mnist.train.num_examples //batch_size  ###采样的次数  


###建立 x，y，权重和偏置项
x= tf.placeholder(tf.float32,[None,784])

y=tf.placeholder(tf.float32,[None,10])

weights= tf.Variable(tf.truncated_normal([lstm_size,n_classes],stddev=0.1))

biases=tf.Variable(tf.constant(0.1,shape=[n_classes]))

##定义神经网络
###输入变量 x，权重和偏置项
def RNN(X,weights,biases):
    ###输入的时候改变数据的格式 按28*28排列提取每个
    inputs=tf.reshape(X,[-1,max_time,n_inputs])
    ###建立RNN核心使用LSTM
    lstm_cell=tf.contrib.rnn.BasicLSTMCell(lstm_size)
    ###然后输出outputs 和 final_state
    outputs,final_state=tf.nn.dynamic_rnn(lstm_cell,inputs,dtype=tf.float32)
    ###找到final_state为 1 的输出并且和他们的weights 相乘，即开门的输出，之后再加上偏置项
    results= tf.nn.softmax(tf.matmul(final_state[1],weights)+biases)
    return results

###产生预测值
prediction= RNN(x,weights,biases)
##计算交叉熵
cross_entropy= tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction,labels=y))

##选择训练的优化器
train_step= tf.train.AdamOptimizer(0.01).minimize(cross_entropy)

##计算正确率
correct_prediction=tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))
##计算准确率
acc=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

##初始化变量
init= tf.global_variables_initializer()

##开始写计算图
with tf.Session() as sess:
    sess.run(init)
    for i in range(6):
        for batch in range(n_batch):
            ##得到每一个batch的x和y
            batch_xs,batch_ys =mnist.train.next_batch(batch_size)
            ##将x和y输入模型
            sess.run(train_step,feed_dict= {x:batch_xs,y:batch_ys})
        ##计算准确率
        acc1=sess.run(acc,feed_dict={x:mnist.test.images,y:mnist.test.labels})
        ##打印
        print('Iter'+str(i)+',Testing acc'+str(acc1))




Extracting MNIST_data/train-images-idx3-ubyte.gz


Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


Iter0,Testing acc0.8946


Iter1,Testing acc0.9235


Iter2,Testing acc0.9402


Iter3,Testing acc0.9183


Iter4,Testing acc0.9455


Iter5,Testing acc0.9356


In [ ]:
_ys